Para el correcto funcionamiento de este fichero es necesario crear dentro de Colab las carpetas ods, csv, txt y pdf. Dentro de las carpetas pdf y ods deberán estar todos los ficheros a transformar

In [1]:
import pandas as pd
!pip install tika
from tika import parser
from google.colab import files
!pip install odfpy

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32625 sha256=7ec13b40f1d76b28604dab298722106a2e9474d8be25ad8cbf7f692baf514ee5
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 38.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=b996831c3e17a92644f1b37194ee7fa840e6d0b66946c2b7c92473f187fc20da
  Stored in directory: /root/.cache/pip/wheels/c8/2e/95/90d94fe33903786937f3b8c33dd88807f792359c6424b40469
Successfully built odfpy


In [2]:
def init():

  # opening pdf file
  parsed_pdf = parser.from_file(ruta + "pdf/" + document + ".pdf")

  # saving content of pdf
  # you can also bring text only, by parsed_pdf['text']
  # parsed_pdf['content'] returns string
  data = parsed_pdf['content']

  # writing to file
  with open(ruta + "txt/" + document + ".txt", 'w') as f:
      f.write(data)

  # reading from file
  file1 = open(ruta + "txt/" + document + ".txt", 'r')
  Lines = file1.readlines()
  extractPagesProrrogados(pagina, pages, Lines)
  # extractPage([], pages, 3, Lines)
  # extractPage([], pages, 5, Lines)

In [3]:
# Crea listas de cada página
def extractPagesProrrogados(myList, pages, Lines):
  listaSinEspacios = []
  # Formateamos y quitamos todas las líneas vacías
  for line in Lines:
    if line != '\n' and line != ' \n':
      listaSinEspacios.append(line)
  Lines = listaSinEspacios
  # Recorremos enviando las páginas a extraer los datos
  for count, line in enumerate(Lines):
    # print(str(count))
    # print(myList)
    almacenar = False
    if (count+1) < len(Lines):
      siguiente = count + 1
    else:
      siguiente = count
    if Lines[siguiente].find("Página") != -1:
      for x in range(1,5):
        if Lines[siguiente+x].find("Objeto del contrato:") != -1 and almacenar == False:
          almacenar = True
      if almacenar:
        pages +=1
        if pages !=0:
          # writeTxt(myList, pages)
          extractDataProrrogados(myList, pages)
          myList = []
    myList.append(line)
  extractDataProrrogados(myList, pages+1)

In [4]:
# Extrae los datos de una página de los contratos prorrogados guardada en una lista de Strings
def extractDataProrrogados(pagina, pages):

  # Inicializamos las variables para poder usarlas en cada llamada e introducirlas en el DataSet tras el bucle For

  numExpediente = objContrato = organismo  = tipoContrato = tipoTramitacion = contratista = " "
  fechaAprobacion = fechaInicio = fechaFin = importe = linea2 =  " "
  fechasAprobacion = []
  fechasInicio = []
  fechasFin = []
  importes = []
  # idObjContrato = idCausa = -1

  # print("Aqui se extraerían los datos de la página " + str(pages))
  for count, value in enumerate(pagina):
    if "Nº Expediente:" in value:
      numExpediente = value[(value.find("Nº Expediente:")) + 15:]
    if "Objeto del contrato:" in value:
      idObjContrato = count - 1
      while ("Página" not in pagina[idObjContrato] and idObjContrato >= 0):
        idObjContrato -= 1
      for x in range(idObjContrato+1, count):
        linea2 = pagina[x]
        objContrato += linea2
      objContrato = objContrato.replace('\n', " ")
      # print(objContrato)
    if "Órgano gestor:" in value:
      organismo = value[(value.find("Órgano gestor:") + 15):(value.find("Nº Expediente:"))]
    if "Tramitación:" in value:
      tipoContrato =  value[: (value.find("Tramitación:"))]
      tipoTramitacion = value[(value.find("Tramitación:") + 13) :]
    if "Importe adjudicación sin I.V.A.:" in value:
        if "Meses:" not in pagina[count+2]:
          contratista = pagina[count+2]
      # Va a necesitar un bucle, todas estas variables
      # fechaAprobacion = fechaInicio = fechaFin = importe = linea2 =  " "

    if "Hasta:" in value:
      if len(value) > 25:
        fechaFin = value[(value.find("Hasta:") + 7) :]
        fechasFin.append(fechaFin)
      else:
        fechaFin = 0
        fechasFin.append(fechaFin)
    if "Desde:" in value:
      if "/" in value:
        fechaInicio = value[(value.find("Desde:") + 7) :]
        fechasInicio.append(fechaInicio)
      else:
        fechaInicio = 0
        fechasInicio.append(fechaInicio)
    if "Duración:" in value:
      indice = count
      if ":" in pagina[count+1]:
        indice = count+1
      if "/" in pagina[indice+1]:
        fechaAprobacion = pagina[indice+1]
        fechasAprobacion.append(fechaAprobacion)
      elif "/" in pagina[indice+2]:
        fechaAprobacion = pagina[indice+2]
        fechasAprobacion.append(fechaAprobacion)
      else:
        fechaAprobacion = " "
        fechasAprobacion.append(fechaAprobacion)
      if "," in pagina[indice+1]:
        importe = pagina[indice+1]
        importes.append(importe)
      elif "," in pagina[indice+2]:
        importe = pagina[indice+1]
        importes.append(importe)
      else:
        importe = " "
        importes.append(importe)
  # Creamos la lista fila con todos los datos de una fila del DataSet e introducimos la fila en el fichero, esto se repite en los x llamadas que se hacen a la función,
  # x equivalente al número de páginas del documento

  for i in range(len(fechasFin)):
    fila = [numExpediente, objContrato, organismo, tipoContrato, tipoTramitacion, fechasAprobacion[i], fechasInicio[i], fechasFin[i], importes[i], contratista]
    prorrogados.loc[len(prorrogados)] = fila
    # prorrogados.to_csv(ruta + "csv/prorrogados_" + str(anyo) + ".csv")
    prorrogados.to_csv(ruta + "csv/prorrogados_" + str(anyo) + ".csv", encoding='utf-8')


In [5]:
# Convierte los archivos ODS a CSV para poder juntar todos en el mismo formato
def ods_to_csv():
  for anyo in range (2019, 2023):
    # print(str(anyo))
    columnas = ["Num. Exp",
              "Objeto Contrato",
              "Organismo",
              "Tipo de Contrato",
              "Tipo Tramitación",
              "Fecha Aprobación",
              "Fecha Inicio",
              "Fecha Fin Prórroga",
              "Importe",
              "Razón Social Adjudicatario/Contratista"]
    columnasODS = [ 'Num. Exp',
                    'Organismo',
                    'Objeto Contrato',
                    "Tipo de Contrato",
                    "Tipo Tramitación",
                    "Importe",
                    "Fecha Aprobación",
                    "Fecha Inicio",
                    "Fecha Fin Prórroga"]

    ruta = "/content/"
    document = "ods/prorrogados_" + str(anyo) + ".ods"

    if anyo == 2019:
      prorrogados = pd.read_excel(ruta + document, engine="odf", header=1)
      prorrogados.columns = columnasODS
      prorrogados.insert(0, "Razón Social Adjudicatario/Contratista", "NaN")
      prorrogados = prorrogados.reindex(columns=columnas)

    if anyo == 2020:
      prorrogados = pd.read_excel(ruta + document, engine="odf", header=1)
      prorrogados.columns = columnasODS
      prorrogados.insert(0, "Razón Social Adjudicatario/Contratista", "NaN")
      prorrogados = prorrogados.reindex(columns=columnas)

    if anyo == 2021:
      prorrogados = pd.read_excel(ruta + document, engine="odf", header=4)
      prorrogados = prorrogados.drop(columns="Unnamed: 2")
      prorrogados = prorrogados.drop(columns="Tipo Incidencia")
      prorrogados = prorrogados.drop(0)
      columnasODS = ['Organismo',
                    'Num. Exp',
                    'Objeto Contrato',
                    "Tipo de Contrato",
                    "Tipo Tramitación",
                    "Importe",
                    "Fecha Aprobación",
                    "Fecha Inicio",
                    "Fecha Fin Prórroga"]
      prorrogados.columns = columnasODS
      prorrogados = prorrogados.reindex(columns=columnas)

    if anyo == 2022:
      prorrogados = pd.read_excel(ruta + document, engine="odf", header=2)
      prorrogados = prorrogados.drop(columns="Unnamed: 1")
      prorrogados = prorrogados.drop(0)
      columnasODS = ['Num. Exp',
                    'Objeto Contrato',
                    "Tipo de Contrato",
                    "Fecha Aprobación",
                    "Fecha Fin Prórroga",
                     "Importe",
                     "Razón Social Adjudicatario/Contratista"]
      prorrogados.columns = columnasODS
      prorrogados.insert(0, "Organismo", "NaN")
      prorrogados.insert(0, "Tipo Tramitación", "NaN")
      prorrogados.insert(0, "Fecha Inicio", "NaN")
      prorrogados = prorrogados.reindex(columns=columnas)

    # prorrogados.to_csv(ruta + "csv/prorrogados_" + str(anyo) + ".csv")
    prorrogados.to_csv(ruta + "csv/prorrogados_" + str(anyo) + ".csv", encoding='utf-8')

In [6]:
# Une los contratos de todos los años, añadiendo una columna "Año" en un archivo CSV
def unir_prorrogados():
  union = pd.DataFrame()
  for anyo in range (2015, 2023):
    ruta = "/content/"
    document = "csv/prorrogados_" + str(anyo) + ".csv"

    prorrogados = pd.read_csv(ruta+document)
    prorrogados = prorrogados.drop(columns="Unnamed: 0")

    prorrogados.insert(len(columnas), "Año", anyo)
    union = pd.concat([union, prorrogados])


    # display(union)
  # union.to_csv(ruta + "prorrogados" + ".csv")
  union.to_csv(ruta + "prorrogados" + ".csv", encoding='utf-8')

In [7]:
# Ejecutar todo
for anyo in range (2015, 2021):
# anyo = 2015
  pagina = []
  pages = -1
  columnas = ["Num. Exp",
              "Objeto Contrato",
              "Organismo",
              "Tipo de Contrato",
              "Tipo Tramitación",
              "Fecha Aprobación",
              "Fecha Inicio",
              "Fecha Fin Prórroga",
              "Importe",
              "Razón Social Adjudicatario/Contratista"]
  prorrogados = pd.DataFrame(columns = columnas)
  ruta = "/content/"
  document = "contratos-prorrogados-" + str(anyo);
  init()

ods_to_csv()

2023-06-29 07:53:04,519 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2023-06-29 07:53:04,768 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2023-06-29 07:53:04,995 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2023-06-29 07:53:10,010 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [8]:
unir_prorrogados()

In [9]:
for x in range (2015, 2023):
  files.download("/content/csv/prorrogados_" + str(x) + ".csv")
files.download("/content/prorrogados" + ".csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>